In [1]:
import numpy as np
from matplotlib import pyplot as plt

import sys
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(14)

from sim_functions import *
from positioning import *
from system_parameters import *
from optimum_solution import *

In [2]:
# System constant parameters

num_ue, num_ap, num_ch = 4, 4, 1
                    
tot_band = 100e6 # [W]
env_constant = 10e-4
pl_constant = 4 

cov_side = 100 # [m]

pmin, pmax, step = 0, 30, 1 # [dBm]

In [4]:
# Study case dataset

ap_pos = ap_positions(num_ap, cov_side)

ue_pos = np.array([[22.538 + 20.333j], 
                   [56.679 + 32.188j], 
                   [76.551 + 14.688j], 
                   [26.595 + 70.239j]])

dis_matrix = distance(ue_pos, ap_pos)

shadowing_matrix = np.array([[5.3434e-2, 2.8731e-1, 1.9691e-2, 7.3013e-1], 
                             [3.2318, 1.5770, 2.6449e-1, 5.6379], 
                             [6.1470e-3, 1.1424, 2.6826e-1, 4.5709], 
                             [1.3485e-1, 4.6690e-1, 7.8250e-1, 1.6742]])

# I dunno why rayleigh_matrix has to have shape (num_ch, num_ue, num_ap) but for operation I'm doing, it works.

rayleigh_matrix = np.array([[1.248699, 3.248041, 0.772754, 0.708962], 
                            [0.498887, 0.104890, 0.647280, 0.940906], 
                            [0.382966, 0.682700, 1.891256, 0.327100], 
                            [0.065737, 0.649500, 1.981107, 1.259538]]).reshape(((num_ch, num_ue, num_ap)))

In [5]:
# Simulation

# Channel matrix is constant as distance, shadowing and fast fading matrixes are constant
channel_matrix = get_channel(dis_matrix, shadowing_matrix, np.sqrt(rayleigh_matrix))

# Set of all vectors of power possible in system
power_set = available_power_vec(pmin, pmax, step, num_ue)

print(len(power_set))

# Noise power in system is constant
n_power = get_noise(tot_band, num_ch)

923521


In [6]:
maxmin = 0
summax = 0
maxprod = 0

for vec in range(len(power_set)):

    # Update the values in order to maximize each objective function
    maxmin = np.max([maxmin, np.min(get_sinr(power_set[vec], channel_matrix, n_power))])
    summax = np.max([summax, np.sum(get_sinr(power_set[vec], channel_matrix, n_power))])
    maxprod = np.max([maxprod, np.sum(get_sinr(power_set[vec], channel_matrix, n_power))*np.min(get_sinr(power_set[vec], channel_matrix, n_power))]) 

In [8]:
print('maxmin: ', lin2db(maxmin))
print('maxsum: ', lin2db(summax))
print('maxprod: ', lin2db(maxprod))

maxmin:  -2.9705022802165306
maxsum:  68.64328358541006
maxprod:  47.45551681824939
